In [1]:
import becquerel as bq
import pandas as pd
from becquerel import Spectrum
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import math as m

In [2]:
def gaussian(x):
    return (spec.livetime * amp /(m.sqrt(2*m.pi)*sigma)) * m.exp(- ((x-mu)**2) / (2*sigma**2))

In [6]:
spec = Spectrum.from_file(r'C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-30min.Spe') #import spectrum from file
len(spec)
bg = Spectrum.from_file(r'C:/Users/Daniel/Desktop/Radwatch/FishSamples2019/Background_Spectrum_12_20_19.Spe'
)

SpeFile: Reading file C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-30min.Spe
SpeFile: Reading file C:/Users/Daniel/Desktop/Radwatch/FishSamples2019/Background_Spectrum_12_20_19.Spe


In [7]:
#test spectrum functions
print(spec.start_time)
print(spec.stop_time)
print(spec.realtime)
print(spec.livetime)

2019-11-13 16:22:08
2019-11-13 16:52:41
1833.0
1800.0


In [8]:
#input of sources
#energy and activities

#22 sources from Tyler's efficiency csv
#source_energy = np.array([661.657,59.54,238.6,583.2,2614.5,80.8983,276.38,302.85,356.017,383.851,121.7817,244.6975,
#344.2785,411.1163,778.904,867.373,964.079,1085.869,1112.069,1212.948,1299.14,1408.006]);
#22 activity placeholders
#source_activity = np.linspace(1,22,22);

#specific for this spectrum
#https://drive.google.com/drive/u/0/folders/0BwU6Sr8Jh6Sgd2dabnZ6N1ZxMEk
# K-40, Bi-214, Tl-208
# indices 8140, 9835, 14574
source_energy = np.array([1460, 609, 2614]);
source_activity = np.array([1, 2, 3]);

In [12]:
counts = spec.counts_vals
energies = spec.bin_centers_kev

#function to find index of closest energy where a is the desired energy and n is 
def f_near(a,a0):
    idx = np.abs(a - a0).argmin()
    return idx

In [15]:
bgspec = np.subtract(spec, bg)

NotImplementedError: Addition/subtraction for arbitrary calibrated spectra not implemented

In [14]:
#test plotting
iso_idx = f_near(energies,source_energy[1])
roi_low = iso_idx - 50
roi_high = iso_idx + 50
#plot_counts = spec.counts_vals[roi_low:roi_high]
plot_counts = bgspec.cps_vals[roi_low:roi_high]
plot_energies = spec.energies_kev[roi_low:roi_high]
plt.plot(plot_energies,plot_counts)
#plt.plot(spec.energies_kev,spec.counts_vals)
plt.yscale('log')
np.argmax(counts)
np.sum(plot_counts)

NameError: name 'bgspec' is not defined

In [ ]:
model = ['gauss','line','erf']
fit = bq.core.fitting.Fitter(model, x=bgspec.bin_indices, y=bgspec.cps_vals, y_unc=bgspec.cps_uncs)
idx = f_near(energies,609)
fit.set_roi(idx-100, idx+100)
fit.fit()

#fit.results.params.pretty_print()

fit.custom_plot()

In [ ]:
fit.result.params.pretty_print()

In [ ]:
#calculate integral of fitted Gaussian
amp = fit.result.params['gauss_amp'].value
mu = fit.result.params['gauss_mu'].value
sigma =fit.result.params['gauss_sigma'].value

integral = integrate.quad(gaussian, idx-100, idx+100)
print(integral)

In [ ]:
#Now doing it iteratively through all energies
integrals = []

for n in source_energy:
    model = ['gauss','line','erf']
    fit = bq.core.fitting.Fitter(model, x=spec.bin_indices, y=spec.cps_vals, y_unc=spec.cps_uncs)
    idx = f_near(energies,n)
    print(idx)
    fit.set_roi(idx-100, idx+100)
    fit.fit()
    amp = fit.result.params['gauss_amp'].value
    mu = fit.result.params['gauss_mu'].value
    sigma = fit.result.params['gauss_sigma'].value
    integral = integrate.quad(gaussian, idx-100, idx+100)
    integrals.append(integral[0])

print(integrals)

In [ ]:
#Testing 1764
idx = f_near(energies,1764)
print(idx)
plot_counts = spec.counts_vals[idx-40:idx+40]
plot_energies = spec.energies_kev[idx-40:idx+40]
plt.plot(plot_energies,plot_counts)
plt.yscale('log')

In [ ]:
model = ['gauss','line','erf']
fit = bq.core.fitting.Fitter(model, x=spec.bin_indices, y=spec.cps_vals, y_unc=spec.cps_uncs)
idx = f_near(energies,1460)
fit.set_roi(idx-100, idx+100)
fit.fit()

amp = fit.result.params['gauss_amp'].value
mu = fit.result.params['gauss_mu'].value
sigma = fit.result.params['gauss_sigma'].value
integral = integrate.quad(gaussian, idx-100, idx+100)

print()
print(integral)
fit.custom_plot()

In [ ]:
#Testing 2614
idx = f_near(energies,2614)
print(idx)
plot_counts = spec.counts_vals[idx-40:idx+40]
plot_energies = spec.energies_kev[idx-40:idx+40]
plt.plot(plot_energies,plot_counts)
plt.yscale('log')

In [4]:
fit.set_roi(idx-40, idx+40)
fit.fit()
fit.params #type is lmfit.parameter.Parameters
fit.params.pretty_print()
fit.custom_plot()
amp = fit.result.params['gauss_amp'].value
mu = fit.result.params['gauss_mu'].value
sigma = fit.result.params['gauss_sigma'].value
integral = integrate.quad(gaussian, idx-100, idx+100)

print(integral)

NameError: name 'fit' is not defined

In [5]:
source_isotopes = np.array(['K-40','Bi-214','Tl-208']);
source_energy = np.array([1460, 1764, 2614]);
source_activity = np.array([1, 2, 3]);

efficiencies = []
for x in range(0,len(source_activity)):
    iso = bq.Isotope(source_isotopes[x])
    efficiency = integrals[x]/source_activity[x]
    efficiencies = np.append(efficiencies,efficiency)


NameError: name 'integrals' is not defined